<img src="uva_seal.png">  

## MLlib Regression

### University of Virginia
### DS 5559: Big Data Analytics
### Last Updated: Sep 15, 2019

---  


### SOURCES
Learning Spark, Chapter 11: Machine Learning with MLlib

*Details on regularization equation*  
https://spark.apache.org/docs/1.5.2/ml-linear-methods.html

https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

https://spark.apache.org/docs/latest/mllib-ensembles.html#random-forests

http://spark.apache.org/docs/latest/ml-classification-regression.html#linear-support-vector-machine


### OBJECTIVES
- Introduction to major regression models in MLlib

### CONCEPTS

- Linear regression

---

**Introduction to Regression**

Regression is another common form of supervised learning
The response variable in a regression problem is quantitative or continuous  

Earlier, we discussed the classification problem where the response variable is discrete

Several of the models we discussed for classification also have regression counterparts, including:

- Support vector machines  
- Tree-based methods like random forests and gradient-boosted trees  

To implement the regression counterpart, the same package is loaded but a different method is called.

**Linear Regression**

Linear regression is the most fundamental model used in regression.

Model assumes a linear relationship between a set of explanatory variables $X$ (aka features, factors, predictors, independent variables) and a scalar response variable $Y$.

Linear regression models are most often fit using the *ordinary least squares* (*OLS*) approach.  

More recently and especially in machine learning, an additional regularization term is added to the loss function. Examples include:

- ridge regression ($L^2$-norm penalty)
- lasso ($L^1$-norm penalty)

**Linear Regression Implementation**  
Method `LinearRegressionWithSGD` is used to train the model.  

**Linear Regression Example: load data/train model/predict**

In [1]:
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
        .master("local") \
        .appName("mllib_classifier") \
        .getOrCreate()
sc = spark.sparkContext

In [10]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

data = sc.textFile("lpsa.data")
data.take(2)

In [11]:
# Parse the data
def parsePoint(line):
    values = [float(x) for x in line.replace(',', ' ').split(' ')]
    return LabeledPoint(values[0], values[1:])

In [13]:
parsedData = data.map(parsePoint)
parsedData.take(2)

In [14]:
# Build the model
model = LinearRegressionWithSGD.train(parsedData, iterations=10, step=0.00000001)

In [16]:
# Evaluate the model on training data
valuesAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()

print("Mean Squared Error = " + str(MSE))

Next we show function calls to train various models, comparing  classification and regression.  Code is minimal, and the functions and parameters are different.  

**Random Forest Implementation**

*Classification Model Training*


In [ ]:
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=1000, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=5, maxBins=32)

*Regression Model Training*

In [ ]:
model = RandomForest.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    numTrees=1000, featureSubsetStrategy="auto",
                                    impurity='variance', maxDepth=4, maxBins=32)

**Gradient-Boosted Trees Implementation**

<span style="color:red"> Note that Spark GBTs support binary classification, but not multiclass classification. </span> 

*Classification Model Training*

In [ ]:
model = GradientBoostedTrees.trainClassifier(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=1000)

*Regression Model Training*

In [ ]:
model = GradientBoostedTrees.trainRegressor(trainingData,
                                            categoricalFeaturesInfo={}, numIterations=3)

**Linear SVM Implementation**

By default, linear SVMs are trained with an $L2$ regularization.  $L1$ regularization is also supported.

*Classification Model Training*

In [ ]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

*Regression Model Training*

In [ ]:
lsvm = SVMWithSGD.train(parsedData, iterations=100)

**Performance Measurement**

For the classification problem, we discussed measuring performance in terms of misclassification error.
For the regression problem, a common metric used for performance measurement is *Mean Squared Error.*

**Measuring Mean Squared Error (MSE) for the Regression Problem**

In [ ]:
# compute squared errors of each point and average

testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(testData.count())

Compare this with the computation of misclassification error for the Classification Problem

In [ ]:
# compute fraction of points where predicted and actual labels disagree

testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())

**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) Modify the Linear Regression example in the cells below to fit and evaluate these two models:  

i. Lasso Regression

ii. Ridge Regression

2) Think of at least one real-world example of when you would need to implement each of the following tasks:  
- regression
- binary classification
- multiclass classification
- multilabel classification

If you are not sure about the difference between multiclass and multilabel, here is one resource:

https://scikit-learn.org/stable/modules/multiclass.html